In [6]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from time import sleep
# from dotenv import load_dotenv
# load_dotenv()
# import os
# bearer = os.getenv("TWITTERBEARER")

In [2]:
market = pd.read_csv("./data/crypto_test.csv")

In [4]:
market.sort_values("date",inplace=True)

In [5]:
market

,Unnamed: 0,date,btc,eth
0,0,2021-01-01 00:00:59.977129,29026.97,738.90
1,1,2021-01-01 00:01:58.468112,29079.56,738.90
2,2,2021-01-01 00:01:59.062990,29079.56,740.33
3,3,2021-01-01 00:02:58.302087,29053.76,740.33
4,4,2021-01-01 00:02:58.382793,29053.76,739.48
...,...,...,...,...
199994,199994,2021-03-11 10:42:58.686812,55168.19,1753.06
199995,199995,2021-03-11 10:42:59.101644,55168.19,1755.44
199996,199996,2021-03-11 10:43:55.651832,55224.00,1755.44
199997,199997,2021-03-11 10:44:59.910494,55110.93,1755.44


In [9]:
market["date"] = pd.to_datetime(market["date"])
market["date"]

0        2021-01-01 00:00:59.977129
1        2021-01-01 00:01:58.468112
2        2021-01-01 00:01:59.062990
3        2021-01-01 00:02:58.302087
4        2021-01-01 00:02:58.382793
                    ...            
199994   2021-03-11 10:42:58.686812
199995   2021-03-11 10:42:59.101644
199996   2021-03-11 10:43:55.651832
199997   2021-03-11 10:44:59.910494
199998   2021-03-11 10:45:59.982523
Name: date, Length: 199999, dtype: datetime64[ns]

In [15]:
balance = 200
order = {}
tracking = market.iloc[0]["btc"]
status = "entry"
start = market["date"].iloc[0]
end = market["date"].iloc[-1]
date = start
signal = 0.05
req = 0.1
retrack_days = 1
trades = []
tracking_time = 0
while date < end:
    try:
        ## pull current prices
        current_market = market[market["date"]>=date].iloc[0]
        current_price = current_market["btc"]
        current_date = current_market["date"]
        ## entry order
        if len(order.keys()) < 1 and balance > 0:
            ## successful entry
            if (current_price - tracking) / tracking <= -1 * signal:
                order = {"buy_date":current_date,"buy_price":current_price,"amount": float(balance/current_price)}
                balance = 0
                date = current_date + timedelta(minutes=1)
                status = "buy"
            ## refresh tracking price
            else:
                if tracking_time >= 2880:
                    tracking = current_price
                    print(date,tracking)
                    tracking_time = 0
                    date = current_date + timedelta(minutes=1)
                else:
                    tracking_time += 1
                    date = current_date + timedelta(minutes=1)
        ## sell order
        else:
            ## successful sell order
            if "buy_price" in order.keys() and balance < 1:
                if (current_price - tracking) / tracking >= req:
                    order["sell_date"] = current_date
                    order["sell_price"] = current_price
                    balance = order["amount"] * order["sell_price"]
                    tracking = current_price
                    trades.append(order)
                    print(order)
                    order = {}
                ## requirement not met
                else:
                    date = current_date + timedelta(minutes=1)
            else:
                break
    except Exception as e:
        print(date,str(e))

2021-01-03 17:07:59.860246 32785.37
{'buy_date': Timestamp('2021-01-04 09:34:59.867761'), 'buy_price': 31116.62, 'amount': 0.006427433313772511, 'sell_date': Timestamp('2021-01-06 20:55:59.798936'), 'sell_price': 36096.77}
2021-01-08 22:14:59.739615 39937.17
{'buy_date': Timestamp('2021-01-10 17:35:59.916980'), 'buy_price': 37765.94, 'amount': 0.006143355150635311, 'sell_date': Timestamp('2021-02-08 13:24:59.991773'), 'sell_price': 44280.1}
2021-02-09 11:19:59.603813 46082.53
2021-02-12 04:51:59.381061 47007.0
2021-02-14 21:59:59.616469 48871.57
{'buy_date': Timestamp('2021-02-15 02:21:59.942011'), 'buy_price': 46063.99, 'amount': 0.00590544545545548, 'sell_date': Timestamp('2021-02-19 15:44:59.924929'), 'sell_price': 53796.0}
2021-02-22 05:01:59.586520 55950.28


In [17]:
t = pd.DataFrame(trades)
t.iloc[-1]["sell_price"] * t.iloc[-1]["amount"]

317.68934372168303

In [ ]:
##check account balance
##check existing orders
##check tracking price
##check status
##if no orders and available balance:
### pull market prices
### input buy_order(tracking_price * (1-req))
### status = buy
###elif order and no available balance:
### if order is buy:
###     input sell_ordeR(tracking_price) * (1*req)
### else:
###.    status = sell
###else if no_order and no available_balance:
### db.send("fcked")
### sleep(5mins)

In [ ]:
https://api.twitter.com/2/tweets/counts/recent?query=ethereum

In [2]:
accounts = "https://api-public.sandbox.exchange.coinbase.com/accounts"
spot_prices = "https://api.exchange.coinbase.com/products/product_id/ticker"
orders = "https://api.exchange.coinbase.com/orders?sortedBy=created_at&sorting=desc&limit=100&status=%5B%27open%27%2C%20%27pending%27%5D"

headers = {"Accept": "application/json"}

response = requests.request("GET", url, headers=headers)

print(response.text)

{"message":"Unauthorized."}


In [ ]:
import requests
### order example
url = "https://api.exchange.coinbase.com/orders"

payload = {
    "profile_id": "default profile_id",
    "type": "limit",
    "side": "buy",
    "stp": "dc",
    "stop": "loss",
    "time_in_force": "GTC",
    "cancel_after": "min",
    "post_only": "false"
}
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json"
}

response = requests.request("POST", url, json=payload, headers=headers)

print(response.text)